In [25]:
import re
import numpy as np
import pandas as pd

def process_energy_file(month_name):
    input_filename = f"{month_name}_combined.xlsx"
    output_filename = f"{month_name}_hourly_energy_output.csv"
    
    print(f"--- Processing {input_filename}... ---")

    try:
        # read file as strings for robust parsing
        df = pd.read_excel(input_filename, dtype=str)
    except FileNotFoundError:
        print(f"WARNING: File '{input_filename}' not found. Skipping.")
        return pd.DataFrame() # Return an empty DataFrame if file is missing

    records = []
    current_date = None

    time_pattern = re.compile(r'(\d{2}:\d{2})')      # matches times like 06:15
    date_pattern = re.compile(r'Date\s*:\s*(\d{2}/\d{2}/\d{4})', re.IGNORECASE)
    number_pattern = re.compile(r'-?\d+\.\d+|-?\d+')

    for idx, row in df.iterrows():
        # join row into a single string for searching
        row_str = ' '.join(row.dropna().astype(str).values)

        # detect date line like "Date : 01/04/2023"
        date_match = date_pattern.search(row_str)
        if date_match:
            current_date = date_match.group(1)
            continue

        if current_date is None:
            # skip lines before the first date
            continue

        # try to extract the first time (prefer 'From' time)
        time_match = time_pattern.search(row_str)
        if not time_match:
            # no time on this row — skip
            continue
        time_str = time_match.group(1)  # e.g. '06:15'

        # Extract numeric tokens from the row string
        nums = [float(x) for x in number_pattern.findall(row_str)]

        # Heuristic to find Total MW
        mw_value = None
        if len(nums) >= 2:
            mw_value = nums[-2]
        elif len(nums) == 1:
            mw_value = nums[0]

        if mw_value is None or np.isnan(mw_value):
            continue

        # compute energy for this 15-min block (MWh)
        energy_block_mwh = mw_value * 0.25

        # compose full timestamp
        try:
            ts = pd.to_datetime(f"{current_date} {time_str}", dayfirst=True, format="%d/%m/%Y %H:%M")
        except Exception:
            ts = pd.to_datetime(f"{current_date} {time_str}", dayfirst=True)

        records.append({
            'timestamp': ts,
            'Total_MW': mw_value,
            'Block_Energy_MWh': energy_block_mwh
        })

    if not records:
        print(f"WARNING: No data parsed from '{input_filename}'.")
        return pd.DataFrame()

    # create DataFrame of blocks
    blocks_df = pd.DataFrame(records)
    blocks_df = blocks_df.set_index('timestamp').sort_index()

    # resample to hourly
    hourly = blocks_df.resample('h').agg({
        'Block_Energy_MWh': 'sum',    # total energy produced in that hour (MWh)
        'Total_MW': ['mean', 'max']   # average MW and max MW during that hour
    })

    # flatten MultiIndex columns
    hourly.columns = ['MWh', 'Avg_MW', 'Max_MW']
    hourly = hourly.reset_index()

    # Save individual monthly results
    #hourly.to_csv(output_filename, index=False)
    print(f"Successfully processed and saved '{output_filename}'")
    
    return hourly

# --- Main script execution ---
if __name__ == "__main__":
    
    # List of months to process
    months_to_process = [
        'April', 'May', 'June', 'July', 'August',
        'September', 'October', 'November', 'December'
    ]

    all_dataframes = []
    for month in months_to_process:
        monthly_df = process_energy_file(month)
        if not monthly_df.empty:
            all_dataframes.append(monthly_df)

    # Combine all the monthly data into a single DataFrame
    if all_dataframes:
        final_combined_df = pd.concat(all_dataframes, ignore_index=True)
        
        print("\n--- All months combined successfully! ---")
        #final_combined_df.to_csv('all_months_hourly_energy.csv', index=False)
        print("Final combined data saved to 'all_months_hourly_energy.csv'")
        print("\nFinal DataFrame Info:")
        print(final_combined_df.info())
    else:
        print("\nNo data was processed. The final DataFrame is empty.")


--- Processing April_combined.xlsx... ---
Successfully processed and saved 'April_hourly_energy_output.csv'
--- Processing May_combined.xlsx... ---
Successfully processed and saved 'May_hourly_energy_output.csv'
--- Processing June_combined.xlsx... ---
Successfully processed and saved 'June_hourly_energy_output.csv'
--- Processing July_combined.xlsx... ---
Successfully processed and saved 'July_hourly_energy_output.csv'
--- Processing August_combined.xlsx... ---
Successfully processed and saved 'August_hourly_energy_output.csv'
--- Processing September_combined.xlsx... ---
Successfully processed and saved 'September_hourly_energy_output.csv'
--- Processing October_combined.xlsx... ---
Successfully processed and saved 'October_hourly_energy_output.csv'
--- Processing November_combined.xlsx... ---
Successfully processed and saved 'November_hourly_energy_output.csv'
--- Processing December_combined.xlsx... ---
Successfully processed and saved 'December_hourly_energy_output.csv'

--- All m